In [18]:
# import pandas as pd
# import numpy as np

# # Load your dataset (901 rows)
# df = pd.read_csv("../water_dataset/water_quality.csv")

# # Target total rows
# target_rows = 4001
# current_rows = df.shape[0]
# extra_rows = target_rows - current_rows

# # Ensure 'date' is datetime
# df["date"] = pd.to_datetime(df["date"])

# # Pick numeric columns (to add noise)
# numeric_cols = df.select_dtypes(include=[np.number]).columns

# # Start generating new rows
# synthetic_rows = []
# last_date = df["date"].max()

# for i in range(extra_rows):
#     # Randomly pick a row as base
#     row = df.sample(1).copy().reset_index(drop=True)
    
#     # Add small noise to numeric columns
#     for col in numeric_cols:
#         noise = np.random.normal(0, 0.05 * df[col].std())  # 5% noise
#         row[col] = row[col] + noise
    
#     # Generate next date (daily increment)
#     new_date = last_date + pd.Timedelta(days=i+1)
#     row["date"] = new_date
    
#     synthetic_rows.append(row)

# # Combine original + synthetic
# df_synthetic = pd.concat([df] + synthetic_rows, ignore_index=True)

# # Save expanded dataset
# df_synthetic.to_csv("water_quality_synthetic.csv", index=False)

# print("✅ Synthetic dataset created with", df_synthetic.shape[0], "rows")
# print("📅 Date range:", df_synthetic["date"].min(), "→", df_synthetic["date"].max())


In [19]:
import pandas as pd
patient_data = pd.read_csv("../Datasets/health_dataset/ecg_data.csv")
water_data = pd.read_csv("../Datasets/water_dataset/water_quality_synthetic.csv")

In [20]:
water_data.head()

,date,temperature_C,turbidity_NTU,pH,DO_mg_L,BOD_mg_L,COD_mg_L,nitrate_mg_L,phosphate_mg_L,TDS_mg_L,conductivity_uS_cm,fecal_coliform_CFU_100mL,WQI
0,2010-01-01,22.993185,5.516143,7.520389,7.906427,2.735397,13.885689,1.968090,0.551685,261.190577,619.903396,49.772302,8.458280
1,2010-02-01,24.104013,7.048402,7.394924,7.910389,2.643025,12.059508,2.621924,0.543875,261.148854,671.353870,50.974185,7.266859
2,2010-03-01,25.775834,7.640319,7.321058,7.860950,3.519859,11.877982,3.147997,0.668651,269.189939,689.367677,44.407779,2.568956
3,2010-04-01,26.215549,6.792833,7.186963,7.355597,2.428056,12.540367,3.015166,0.628819,218.568254,571.605619,24.495494,8.083391
4,2010-05-01,22.320496,2.673462,7.065185,8.019599,2.458800,11.160989,2.424284,0.366071,168.266876,517.677840,26.127145,15.988587


In [21]:
print("Patient Data:")
print(patient_data.shape)
print(patient_data.head(), "\n")

print("Water Data:")
print(water_data.shape)
print(water_data.head(), "\n")



Patient Data:
(4000, 10)
               Name  Age ECG Category  Heart Rate  Systolic BP  Diastolic BP  \
0     Rajinder Brar   56  Bradycardia          55          100           103   
1     Gurpreet Gill   55       Normal          78          147           103   
2   Baljinder Bajwa   39       Normal          97          135            84   
3  Paramjit Dhillon   57       Normal          75          118            64   
4   Baljinder Sidhu   46       Normal          99          170            81   

   Glucose  SpO2  Body Temperature  Gender  
0      158    96              38.8  Female  
1      160    96              38.0    Male  
2      112    88              36.6  Female  
3      122    92              37.1  Female  
4      103    87              38.7  Female   

Water Data:
(4000, 13)
         date  temperature_C  turbidity_NTU        pH   DO_mg_L  BOD_mg_L  \
0  2010-01-01      22.993185       5.516143  7.520389  7.906427  2.735397   
1  2010-02-01      24.104013       7.048402  

In [22]:
# Reset index so both start from 0
patient_data = patient_data.reset_index(drop=True)
water_data = water_data.reset_index(drop=True)

# Ensure they have same number of rows (otherwise will mismatch)
min_len = min(len(patient_data), len(water_data))
patient_data = patient_data.iloc[:min_len]
water_data = water_data.iloc[:min_len]

# Merge side by side
merged_data = pd.concat([patient_data, water_data], axis=1)

print(merged_data.shape)
print(merged_data.head())


(4000, 23)
               Name  Age ECG Category  Heart Rate  Systolic BP  Diastolic BP  \
0     Rajinder Brar   56  Bradycardia          55          100           103   
1     Gurpreet Gill   55       Normal          78          147           103   
2   Baljinder Bajwa   39       Normal          97          135            84   
3  Paramjit Dhillon   57       Normal          75          118            64   
4   Baljinder Sidhu   46       Normal          99          170            81   

   Glucose  SpO2  Body Temperature  Gender  ...        pH   DO_mg_L  BOD_mg_L  \
0      158    96              38.8  Female  ...  7.520389  7.906427  2.735397   
1      160    96              38.0    Male  ...  7.394924  7.910389  2.643025   
2      112    88              36.6  Female  ...  7.321058  7.860950  3.519859   
3      122    92              37.1  Female  ...  7.186963  7.355597  2.428056   
4      103    87              38.7  Female  ...  7.065185  8.019599  2.458800   

    COD_mg_L  nitrate

In [23]:
import pandas as pd

def label_disease(row):
    """
    Labels a row with a potential disease based on a set of rules
    combining water quality and personal health data.
    Expanded to include more diseases to reduce "Uncertain" classifications.
    """
    try:
        # Typhoid
        # Conditions: High fecal coliform, high temp, high glucose, poor water quality.
        if (
            row["fecal_coliform_CFU_100mL"] > 15 and
            row["Body Temperature"] > 37.8 and
            row["Glucose"] > 95 and
            row["WQI"] < 40
        ):
            return "Typhoid"

        # Cholera
        # Conditions: High organic matter, low pH, very high BP, high temp, very high fecal coliform.
        elif (
            row["BOD_mg_L"] > 3 and
            row["pH"] < 6.5 and
            row["Systolic BP"] > 140 and
            row["Body Temperature"] > 38.2 and
            row["fecal_coliform_CFU_100mL"] > 20
        ):
            return "Cholera"

        # Diarrhea
        # Conditions: High turbidity, low oxygen, fever, high phosphates.
        elif (
            row["turbidity_NTU"] > 4 and
            row["SpO2"] < 96 and
            row["Body Temperature"] > 37.3 and
            row["phosphate_mg_L"] > 0.3
        ):
            return "Diarrhea"

        # Hepatitis A
        # Conditions: High chemical oxygen demand, low glucose, fever, high nitrates.
        elif (
            row["COD_mg_L"] > 8 and
            row["Glucose"] < 80 and
            row["Body Temperature"] > 37.5 and
            row["nitrate_mg_L"] > 2
        ):
            return "Hepatitis A"

        # Dysentery
        # Conditions: High nitrates & phosphates, low oxygen, high heart rate.
        elif (
            row["nitrate_mg_L"] > 2.5 and
            row["phosphate_mg_L"] > 0.4 and
            row["SpO2"] < 95 and
            row["Heart Rate"] > 90
        ):
            return "Dysentery"

        # E. coli Infection
        # Conditions: Poor water quality, high conductivity, fever, high fecal coliform.
        elif (
            row["WQI"] < 45 and
            row["conductivity_uS_cm"] > 500 and
            row["Body Temperature"] > 37.6 and
            row["fecal_coliform_CFU_100mL"] > 12
        ):
            return "E. coli Infection"

        # Gastroenteritis
        # Conditions: Moderate contamination, mild fever, elevated heart rate.
        elif (
            row["fecal_coliform_CFU_100mL"] > 8 and
            row["Body Temperature"] > 37.2 and
            row["Heart Rate"] > 85 and
            row["turbidity_NTU"] > 2.5
        ):
            return "Gastroenteritis"

        # Salmonella Poisoning
        # Conditions: High BOD, moderate fever, poor water quality, elevated glucose.
        elif (
            row["BOD_mg_L"] > 2.5 and
            row["Body Temperature"] > 37.4 and
            row["WQI"] < 55 and
            row["Glucose"] > 88
        ):
            return "Salmonella Poisoning"

        # Giardiasis
        # Conditions: High turbidity, low pH, mild symptoms.
        elif (
            row["turbidity_NTU"] > 3 and
            row["pH"] < 6.8 and
            row["Body Temperature"] > 37.1 and
            row["nitrate_mg_L"] > 1.5
        ):
            return "Giardiasis"

        # Cryptosporidiosis
        # Conditions: Moderate water contamination, mild fever, normal BP.
        elif (
            row["WQI"] < 60 and
            row["Body Temperature"] > 37.0 and
            row["Systolic BP"] < 130 and
            row["fecal_coliform_CFU_100mL"] > 6
        ):
            return "Cryptosporidiosis"

        # Shigellosis
        # Conditions: High phosphates, elevated heart rate, moderate fever.
        elif (
            row["phosphate_mg_L"] > 0.25 and
            row["Heart Rate"] > 88 and
            row["Body Temperature"] > 37.2 and
            row["COD_mg_L"] > 5
        ):
            return "Shigellosis"

        # Rotavirus Infection
        # Conditions: Poor water quality, mild fever, young-like symptoms.
        elif (
            row["WQI"] < 50 and
            row["Body Temperature"] > 37.1 and
            row["SpO2"] < 97 and
            row["turbidity_NTU"] > 2
        ):
            return "Rotavirus Infection"

        # Norovirus Infection
        # Conditions: Moderate contamination, normal temperature, high heart rate.
        elif (
            row["fecal_coliform_CFU_100mL"] > 5 and
            row["Body Temperature"] < 37.5 and
            row["Heart Rate"] > 95 and
            row["conductivity_uS_cm"] > 400
        ):
            return "Norovirus Infection"

        # Campylobacter Infection
        # Conditions: High conductivity, moderate fever, elevated nitrates.
        elif (
            row["conductivity_uS_cm"] > 450 and
            row["Body Temperature"] > 37.3 and
            row["nitrate_mg_L"] > 1.8 and
            row["pH"] < 7.0
        ):
            return "Campylobacter Infection"

        # Water-related Skin Infection
        # Conditions: High conductivity, normal temp, poor water quality.
        elif (
            row["conductivity_uS_cm"] > 600 and
            row["Body Temperature"] < 37.2 and
            row["WQI"] < 45 and
            row["pH"] > 8.0
        ):
            return "Water-related Skin Infection"

        # Mild Water Intolerance
        # Conditions: Slightly poor water, mild symptoms.
        elif (
            row["WQI"] < 65 and
            row["Body Temperature"] > 36.8 and
            row["Heart Rate"] > 80 and
            row["SpO2"] > 95
        ):
            return "Mild Water Intolerance"

        # Dehydration Stress
        # Conditions: High heart rate, normal temp, poor hydration indicators.
        elif (
            row["Heart Rate"] > 85 and
            row["Body Temperature"] < 37.0 and
            row["conductivity_uS_cm"] > 350 and
            row["Glucose"] > 85
        ):
            return "Dehydration Stress"

        # Environmental Stress
        # Conditions: Moderate water issues, mild physiological response.
        elif (
            row["WQI"] < 70 and
            row["Body Temperature"] > 36.9 and
            (row["Heart Rate"] > 82 or row["Systolic BP"] > 125)
        ):
            return "Environmental Stress"

        # No Disease / Healthy
        # Conditions: Excellent water quality and healthy vitals.
        elif (
            row["WQI"] > 75 and
            row["Body Temperature"] < 37.0 and
            row["SpO2"] > 97 and
            row["Heart Rate"] < 80 and
            row["Glucose"] < 90
        ):
            return "Healthy"

        # Mild Risk
        # Conditions: Good water quality but some minor indicators.
        elif (
            row["WQI"] > 60 and
            row["Body Temperature"] < 37.2 and
            row["SpO2"] > 96
        ):
            return "Mild Risk"

        # Default fallback - should rarely be reached now
        else:
            return "Uncertain"
            
    except (TypeError, KeyError):
        # This will catch errors if a row has missing data (NaN) 
        # or if a column is missing, preventing the script from crashing.
        return "Insufficient Data"

In [24]:
merged_data["Disease_Label"] = merged_data.apply(label_disease,axis=1)

In [25]:
merged_data.to_csv("merged_data.csv")

In [26]:
columns = ["Name","date"]
merged_data = merged_data.drop(columns=columns)

In [27]:
merged_data

,Age,ECG Category,Heart Rate,Systolic BP,Diastolic BP,Glucose,SpO2,Body Temperature,Gender,temperature_C,...,DO_mg_L,BOD_mg_L,COD_mg_L,nitrate_mg_L,phosphate_mg_L,TDS_mg_L,conductivity_uS_cm,fecal_coliform_CFU_100mL,WQI,Disease_Label
0,56,Bradycardia,55,100,103,158,96,38.8,Female,22.993185,...,7.906427,2.735397,13.885689,1.968090,0.551685,261.190577,619.903396,49.772302,8.458280,Typhoid
1,55,Normal,78,147,103,160,96,38.0,Male,24.104013,...,7.910389,2.643025,12.059508,2.621924,0.543875,261.148854,671.353870,50.974185,7.266859,Typhoid
2,39,Normal,97,135,84,112,88,36.6,Female,25.775834,...,7.860950,3.519859,11.877982,3.147997,0.668651,269.189939,689.367677,44.407779,2.568956,Dysentery
3,57,Normal,75,118,64,122,92,37.1,Female,26.215549,...,7.355597,2.428056,12.540367,3.015166,0.628819,218.568254,571.605619,24.495494,8.083391,Cryptosporidiosis
4,46,Normal,99,170,81,103,87,38.7,Female,22.320496,...,8.019599,2.458800,11.160989,2.424284,0.366071,168.266876,517.677840,26.127145,15.988587,Typhoid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,72,Normal,90,180,85,72,91,39.3,Female,22.822361,...,7.652719,3.136017,14.801661,2.635383,0.591130,237.916715,593.994556,43.128231,3.492863,Diarrhea
3996,50,Tachycardia,103,154,93,75,92,35.8,Female,13.420486,...,9.727111,2.313636,13.486259,1.092901,0.290991,154.342661,541.826969,33.339636,18.055828,Norovirus Infection
3997,60,Normal,65,116,107,162,91,38.6,Female,20.787313,...,8.047614,2.213940,11.966064,2.105739,0.425819,190.684862,600.955920,24.327634,12.084082,Typhoid
3998,47,Bradycardia,49,178,109,88,89,38.5,Female,16.455613,...,9.166387,2.949955,15.876295,1.948227,0.365551,158.969784,565.359741,39.579521,11.302585,Diarrhea


In [28]:
mem = merged_data["Disease_Label"].unique()
print(mem)

['Typhoid' 'Dysentery' 'Cryptosporidiosis' 'E. coli Infection' 'Uncertain'
 'Norovirus Infection' 'Diarrhea' 'Dehydration Stress' 'Hepatitis A'
 'Salmonella Poisoning' 'Rotavirus Infection' 'Gastroenteritis'
 'Environmental Stress' 'Mild Water Intolerance' 'Shigellosis']


In [29]:
merged_data["Gender"] = merged_data["Gender"].astype(str).map({
    "Male": 0,
    "Female": 1
})

merged_data["ECG Category"] = merged_data["ECG Category"].astype(str).map({
    "Bradycardia" : 0,
    "Normal" : 1,
    "Tachycardia" : 2,
    "Arrhythmia" : 3
})

merged_data["Disease_Label"] = merged_data["Disease_Label"].astype(str).map({
        "Typhoid": 0,
    "Dysentery": 1,
    "Cryptosporidiosis": 2,
    "E. coli Infection": 3,
    "Uncertain": 4,
    "Norovirus Infection": 5,
    "Diarrhea": 6,
    "Dehydration Stress": 7,
    "Hepatitis A": 8,
    "Salmonella Poisoning": 9,
    "Rotavirus Infection": 10,
    "Gastroenteritis": 11,
    "Environmental Stress": 12,
    "Mild Water Intolerance": 13,
    "Shigellosis": 14

})

In [30]:
merged_data.to_csv("Merged_Data.csv")

In [31]:
merged_data.columns

Index(['Age', 'ECG Category', 'Heart Rate', 'Systolic BP', 'Diastolic BP',
       'Glucose', 'SpO2', 'Body Temperature', 'Gender', 'temperature_C',
       'turbidity_NTU', 'pH', 'DO_mg_L', 'BOD_mg_L', 'COD_mg_L',
       'nitrate_mg_L', 'phosphate_mg_L', 'TDS_mg_L', 'conductivity_uS_cm',
       'fecal_coliform_CFU_100mL', 'WQI', 'Disease_Label'],
      dtype='object')

In [32]:
import joblib
disease_mapping = {i: label for i, label in enumerate(merged_data["Disease_Label"].unique())}
joblib.dump(disease_mapping, "disease_mapping.pkl")


['disease_mapping.pkl']

In [34]:
print(merged_data["Disease_Label"].value_counts())


Disease_Label
0     1559
4      765
6      351
5      303
3      233
7      184
2      136
1      105
12     103
11      76
10      72
8       51
9       36
13      19
14       7
Name: count, dtype: int64
